In [1]:
import random,math,copy,timeit
from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from copy import deepcopy as dc
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection as ms

In [2]:
def random_search(n,dim):
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
    return gens



In [3]:
def evaluate(train_d,train_l,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in train_d])
        kf = ms.KFold(n_splits=5)
        s = 0
        for tr_ix,te_ix in kf.split(al_data):
            s+= KNeighborsClassifier(n_neighbors=5).fit(al_data[tr_ix],train_l[tr_ix]).score(al_data[te_ix],train_l[te_ix])#.predict(al_test_data)
        s/=5
        return s

In [4]:
def logsig(n): return 1 / (1 + math.exp(-n))
def sign(x): return 1 if x > 0 else (-1 if x!=0 else 0)

def BPSO(train_d,train_l,n=20,max_iter=200,w1=0.5,c1=0.5,c2=0.5,vmax=4):
    """
    input:{ 
            Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            max_iter: Number of max iteration, default=300
            dim: Number of feature, default=None
            prog: Do you want to use a progress bar?, default=False
            w1: move rate, default=0.5
            c1,c2: It's are two fixed variables, default=1,1
            vmax: Limit search range of vmax, default=4
            }
    output:{
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    dim=len(train_d[0])
    personal_best=float("-inf")
    global_best=float("-inf")
    gens=random_search(n,dim)
    vel=[[random.random()-0.5 for d in range(dim)] for _n in range(n)]
    one_vel=[[random.random()-0.5 for d in range(dim)] for _n in range(n)]
    zero_vel=[[random.random()-0.5 for d in range(dim)] for _n in range(n)]
    fit=[float("-inf") for i in range(n)]
    personal_best=dc(fit)
    xpersonal_best=dc(gens)
    global_best=max(fit)
    xglobal_best=gens[fit.index(max(fit))]
    gens_dict={tuple([0]*dim):float("-inf")}
    for it in range(max_iter):
        for i in range(n):
            if tuple(gens[i]) in gens_dict:
                score=gens_dict[tuple(gens[i])]
            else:
                score=evaluate(train_d,train_l,gens[i])
                gens_dict[tuple(gens[i])]=score
            fit[i]=score
            if fit[i]>personal_best[i]:#max
                personal_best[i]=dc(fit[i])
                xpersonal_best[i]=dc(gens[i])
        gg=max(fit)
        xgg=gens[fit.index(gg)]
        if global_best<gg:#max
            global_best=dc(gg)
            xglobal_best=dc(xgg)
        oneadd=[[0 for d in range(dim)] for i in range(n)]
        zeroadd=[[0 for d in range(dim)] for i in range(n)]
        c3=c1*random.random()
        dd3=c2*random.random()
        for i in range(n):
            for j in range(dim):
                if xpersonal_best[i][j]==0:
                    oneadd[i][j]=oneadd[i][j]-c3
                    zeroadd[i][j]=zeroadd[i][j]+c3
                else:
                    oneadd[i][j]=oneadd[i][j]+c3
                    zeroadd[i][j]=zeroadd[i][j]-c3
                if xglobal_best[j]==0:
                    oneadd[i][j]=oneadd[i][j]-dd3
                    zeroadd[i][j]=zeroadd[i][j]+dd3
                else:
                    oneadd[i][j]=oneadd[i][j]+dd3
                    zeroadd[i][j]=zeroadd[i][j]-dd3
        one_vel=[[w1*_v+_a for _v,_a in zip(ov,oa)] for ov,oa in zip(one_vel,oneadd)]
        zero_vel=[[w1*_v+_a for _v,_a in zip(ov,oa)] for ov,oa in zip(zero_vel,zeroadd)]
        for i in range(n):
            for j in range(dim):
                if abs(vel[i][j]) > vmax:
                    zero_vel[i][j]=vmax*sign(zero_vel[i][j])
                    one_vel[i][j]=vmax*sign(one_vel[i][j])
        for i in range(n):
            for j in range(dim):
                if gens[i][j]==1:
                    vel[i][j]=zero_vel[i][j]
                else:
                    vel[i][j]=one_vel[i][j]
        veln=[[logsig(s[_s]) for _s in range(len(s))] for s in vel]
        temp=[[random.random() for d in range(dim)] for _n in range(n)]
        for i in range(n):
            for j in range(dim):
                if temp[i][j]<veln[i][j]:
                    gens[i][j]= 0 if gens[i][j] ==1 else 1
                else:
                    pass
    return xglobal_best,xglobal_best.count(1)

In [5]:
def test_score(gen,tr_x,tr_y,te_x,te_y):
    mask=np.array(gen) == 1
    al_data=np.array(tr_x[:,mask])
    al_test_data=np.array(te_x[:,mask])
    return np.mean([KNeighborsClassifier(n_neighbors=5).fit(al_data,tr_y).score(al_test_data,te_y) for i in range(5)])

In [6]:
with open("/home/shubbham28/Downloads/bc/wdbc1.csv") as f:
    x=np.array([[float(d) for d  in data.split(',')] for data in f.read().splitlines()])
with open("/home/shubbham28/Downloads/bc/wdbc2.csv") as f:
    y=np.array([float(data) for data in f.read().splitlines()])
lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y)
train_d, test_d, train_l, test_l = train_test_split(x, training_scores_encoded, test_size=0.25)

In [8]:
k=[1 for r in range(len(x[0]))]
print test_score(k,train_d,train_l,test_d,test_l)
fattr=0
ftest=0.0
flist=[0 for r in range(len(x[0]))]
final_list=[0 for r in range(len(x[0]))]
start=timeit.default_timer()
for i in range(20):
    g,l=BPSO(train_d,train_l,n=20,max_iter=200,w1=0.5,c1=0.5,c2=0.5,vmax=4)
    fattr+=l
    test=test_score(g,train_d,train_l,test_d,test_l)
    ftest+=test
    for j in range(len(flist)):
        if g[j]==1:
            flist[j]+=1
    print("{0}  {1}  {2}  {3:.6f}".format(i+1,"".join(map(str,g)),l,test))
fattr=fattr/20
ftest=ftest/20
end=timeit.default_timer()
time=end-start
final=np.argsort(flist)[::-1][:fattr]
for i in range(len(final)):
    final_list[final[i]]=1
print("{0}  {1}   {2}   {3:.6f}    {4:.4f}".format("Final: ","".join(map(str,final_list)),fattr,ftest,time))

0.8741258741258742
1  010001110111110111111110111111  23  0.902098
2  010010111111110111101110111101  22  0.902098
3  110000001011110101101110000000  13  0.902098
4  110010000011010001011110011010  14  0.902098
5  110011010000110100000110010011  13  0.902098
6  011011000101001111000110001111  16  0.909091
7  110011000001110001011110011100  15  0.902098
8  110010111000111000101110100000  14  0.902098
9  110011100101111100110110010111  19  0.902098
10  110000100011010001011110111011  16  0.902098
11  010001110010111010110110100111  17  0.902098
12  110011011001011001101110111111  20  0.902098
13  110000111110111101011110010100  18  0.902098
14  011010000001001101100110100000  11  0.909091
15  110010001110011000110110111111  18  0.902098
16  110011001000011101111110011001  17  0.902098
17  010001100110010001001110001111  14  0.902098
18  010001101000110110011110011011  16  0.902098
19  110010110110111110010110011111  20  0.902098
20  110000101001011010000110001101  13  0.902098
Final:   1

In [15]:
k=[1 for r in range(len(x[0]))]
print test_score(k,train_d,train_l,test_d,test_l)
fattr=0
ftest=0.0
flist=[0 for r in range(len(x[0]))]
final_list=[0 for r in range(len(x[0]))]
start=timeit.default_timer()
for i in range(20):
    g,l=BPSO(train_d,train_l,n=20,max_iter=200,w1=0.5,c1=0.5,c2=0.5,vmax=4)
    fattr+=l
    test=test_score(g,train_d,train_l,test_d,test_l)
    ftest+=test
    for j in range(len(flist)):
        if g[j]==1:
            flist[j]+=1
    print("{0}  {1}  {2}  {3:.6f}".format(i+1,"".join(map(str,g)),l,test))
fattr=fattr/20
ftest=ftest/20
end=timeit.default_timer()
time=end-start
final=np.argsort(flist)[::-1][:fattr]
for i in range(len(final)):
    final_list[final[i]]=1
print("{0}  {1}   {2}   {3:.6f}    {4:.4f}".format("Final: ","".join(map(str,final_list)),fattr,ftest,time))

0.9300699300699302
1  010001111011000001010010000100  11  0.937063
2  010010000101001100110010000111  12  0.937063
3  010001110011000001000010000000  8  0.937063
4  010000010011000101110010100100  11  0.937063
5  010010000111001110000010100000  10  0.937063
6  010010110010101011011110111011  18  0.937063
7  010010000001001000100010000111  9  0.937063
8  010010000001000111110010100010  11  0.937063
9  010011110101000100000010100110  12  0.937063
10  010011001011101111011110111011  20  0.937063
11  010011000101000110010010100000  10  0.937063
12  010010001101000000000010000111  9  0.937063
13  010010011001000001100010100001  10  0.937063
14  010010011011000101100010100001  12  0.937063
15  010000001011001010000010100001  9  0.937063
16  010001010111001011000010000010  11  0.937063
17  010000111101001110010010000000  11  0.937063
18  010000000011001000000010100101  8  0.937063
19  010000101111000011000010000001  10  0.937063
20  110000001110000000100010000100  8  0.937063
Final:   0100100